In [18]:
import pandas as pd
import numpy as np
from sklearn import tree

import datetime
import random

In [19]:
df = pd.read_excel('dane.xlsx')
df_new = df.copy()

df[['Typ oferty']] = np.where(df[['Typ oferty']] == 'Lokata', 1, 0)
df[['Typ klienta']] = np.where(df[['Typ klienta']] == 'Indywidualny', 1, 0)
df[['Waluta']] = np.where(df[['Waluta']] == 'PLN', 1, 0)
X = df[['Typ oferty', 'Długość (miesiące)', 'Typ klienta', 'Waluta']]

In [20]:
y = df[['Oprocentowanie']]
clf_opr = tree.DecisionTreeRegressor()
clf_opr.fit(X, y)

DecisionTreeRegressor()

In [21]:
tmp = df[~df['Maksymalne środki'].isna()]
y = tmp[['Maksymalne środki']]
clf_max = tree.DecisionTreeRegressor()
clf_max.fit(tmp[['Typ oferty', 'Długość (miesiące)', 'Typ klienta', 'Waluta']], y)

DecisionTreeRegressor()

In [22]:
y = df[['Minimalne środki']].fillna(0)
clf_min = tree.DecisionTreeRegressor()
clf_min.fit(X, y)

DecisionTreeRegressor()

In [23]:
name_components = [
    ['Random', 'Super', 'Gold', 'Awesome'],
    ['Bank'],
    ['Polski', 'Inny', '']
]
def generate_name():
    name = ''
    for l in name_components:
        name += (random.choice(l)) + ' '
    return name


intervals = [
    [0.25, 1, 3, 6],
    [3, 6, 12, 24],
    [6, 12, 18],
    [1, 3, 4, 6, 12, 18, 24]
]
single_interval = [0.25, 1, 3, 4, 6, 9, 12, np.NaN]

types = ['Lokata', "Konto oszczędnościowe"]
client = ['Indywidualny', 'Biznesowy']
offer_type = ['', 'Nowi klienci', 'Starzy klienci', 'Nowe środki']
currency = ['PLN', 'EUR', 'USD', 'GBP']

In [24]:
def gen_row(n, d, t, i):
    k = 0 if random.random() > 0.8 else 1
    w = 0 if random.random() > 0.754 else 1
    
    opr = np.round(np.clip(clf_opr.predict(np.array([t, i, k, w]).reshape((1, -1)))[0] + np.random.normal(0, 0.0007), 0.0001, 0.1), 4)
    
    mi = np.round(np.clip(clf_min.predict(np.array([t, i, k, w]).reshape((1, -1)))[0] + np.random.normal(-2000, 1000), 0, 10000000), -2)
    
    ma = np.NaN if random.random() > 0.823 else np.round(np.clip(clf_max.predict(np.array([t, i, k, w]).reshape((1, -1)))[0] + np.random.normal(0, 10000), 0, 10000000), -2)
    
    r = random.choices(offer_type,
                       [50, 1, 1, 1], k=1)[0]
    
    w = 'PLN' if w else random.choice(['EUR', 'GBP', 'USD'])
    
    row = pd.DataFrame({
        'Bank': [n],
        'Nazwa oferty': ['Lokata'] if t else ['Konto oszczędnościowe'],
        'Oprocentowanie': [opr],
        'Typ oferty': ['Lokata'] if t else ['Konto oszczędnościowe'],
        'Długość (miesiące)': [i],
        'Typ klienta': ['Indywidualny'] if k else ['Biznesowy'],
        'Rodzaj oferty': [r],
        'Minimalne środki': [mi],
        'Maksymalne środki': [ma],
        'Waluta': [w],
        'Uwagi': [''],
        'Ważne od': [d]
    })
    
    return row

In [25]:
for _ in range(1000):
    n = generate_name()
    d = datetime.datetime.fromtimestamp(random.randint(1514769289, 1696122889))
    
    times = np.random.randint(1, 6)
    if times > 2:
        for _ in range(times):
            i = random.choices(single_interval,
                   [1, 1, 1, 1, 1, 1, 1, 13], k=1)[0]
            if i == np.NaN:
                t = 0 # konto oszczędnościowe
            else:
                t = 0 if random.random() > 0.8 else 1
        
            row = gen_row(n, d, t, i)
            df_new = pd.concat([df_new, row])
    else:
        for _ in range(times):
            if random.random() > 0.3:
                i = random.choices(single_interval,
                                   [1, 1, 1, 1, 1, 1, 1, 13], k=1)[0]
                if i == np.NaN:
                    t = 0 # konto oszczędnościowe
                else:
                    t = 0 if random.random() > 0.8 else 1
                
                row = gen_row(n, d, t, i)
                df_new = pd.concat([df_new, row])
            else:
                for i in random.choice(intervals):
                    t = 0 if random.random() > 0.8 else 1
                    row = gen_row(n, d, t, i)
                    df_new = pd.concat([df_new, row])
            

/home/krzysztof/Projekty/Supervisionhack3/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/home/krzysztof/Projekty/Supervisionhack3/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/home/krzysztof/Projekty/Supervisionhack3/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/home/krzysztof/Projekty/Supervisionhack3/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/home/krzysztof/Projekty/Supervisionhack3/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not

In [9]:
df_new.reset_index(drop=True).to_excel('dane_generowane.xlsx', index=False)

In [30]:
df_new

,Bank,Nazwa oferty,Oprocentowanie,Typ oferty,Długość (miesiące),Typ klienta,Rodzaj oferty,Minimalne środki,Maksymalne środki,Waluta,Uwagi,Ważne od
0,Bank Handlowy w Warszawie SA,Konto oszczędnościowe,0.0200,Konto oszczędnościowe,NaN,Indywidualny,NaN,NaN,NaN,0,NaN,2023-12-05 00:00:00
1,Bank Handlowy w Warszawie SA,Konto oszczędnościowe,0.0001,Konto oszczędnościowe,NaN,Indywidualny,NaN,NaN,NaN,0,NaN,2023-12-05 00:00:00
2,Bank Handlowy w Warszawie SA,Konto oszczędnościowe,0.0001,Konto oszczędnościowe,NaN,Indywidualny,NaN,NaN,NaN,0,NaN,2023-12-05 00:00:00
3,Bank Handlowy w Warszawie SA,Konto superoszczędnościowe,0.0500,Konto oszczędnościowe,NaN,Indywidualny,NaN,NaN,20000.0,0,Powyżej 20k 2%,2023-12-05 00:00:00
4,Bank Handlowy w Warszawie SA,Konto superoszczędnościowe,0.0005,Konto oszczędnościowe,NaN,Indywidualny,NaN,NaN,20000.0,0,Powyżej 20k 0%,2023-12-05 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Gold Bank Polski,Lokata,0.0425,Lokata,4.0,Indywidualny,,100.0,72100.0,0,,2021-07-14 13:15:45
0,Gold Bank Polski,Lokata,0.0031,Lokata,6.0,Biznesowy,,0.0,124800.0,0,,2021-07-14 13:15:45
0,Gold Bank Polski,Lokata,0.0354,Lokata,12.0,Indywidualny,,500.0,101600.0,0,,2021-07-14 13:15:45
0,Gold Bank Polski,Lokata,0.0028,Lokata,18.0,Indywidualny,,1100.0,NaN,0,,2021-07-14 13:15:45
